In [ ]:
#! uv pip install jupyter requests qdrant-client llama-index llama-index-embeddings-huggingface llama-index-vector-stores-qdrant llama-index-llms-ollama watchdog

# Get the sources

In [ ]:
import os
import subprocess
import yaml
from pathlib import Path

# Path to your sources.yaml file
yaml_file = Path("sources.yaml")

# Load YAML file
with yaml_file.open("r") as f:
    sources = yaml.safe_load(f)

# Base directory for all repos
base_dir = Path("..") / "markdown_docs"

for key, repos in sources.items():
    dir_path = base_dir / key
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"\n📁 Directory: {dir_path}")

    for repo in repos:
        repo_name = Path(repo).stem.replace(".git", "")
        target_dir = dir_path / repo_name

        if target_dir.exists() and (target_dir / ".git").exists():
            print(f"🔄 Updating existing repo: {repo_name}")
            subprocess.run(
                ["git", "pull", "--ff-only"],
                cwd=target_dir,
                check=False
            )
        else:
            print(f"⬇️ Cloning new repo: {repo_name}")
            subprocess.run(
                ["git", "clone", "--depth=1", repo],
                cwd=dir_path,
                check=True
            )


# load the markdowns

In [ ]:
# 🧩 Local RAG Question Answering with Ollama + Qdrant + Markdown files

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Settings,
)
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from qdrant_client import QdrantClient, models
import requests
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

def ensure_model(model_name: str = "llama3.2:latest", host: str = "http://localhost:11434", allow_pull: bool = True):
    """Ensure the given Ollama model exists locally."""
    try:
        resp = requests.get(f"{host}/api/tags")
        resp.raise_for_status()
        tags = [m["name"] for m in resp.json().get("models", [])]
        if model_name in tags:
            print(f"✅ Model '{model_name}' found locally.")
            return True
        elif allow_pull:
            print(f"⬇️ Pulling '{model_name}' from Ollama registry...")
            requests.post(f"{host}/api/pull", json={"name": model_name})
            print(f"✅ Model '{model_name}' pulled successfully.")
            return True
        else:
            print(f"❌ Model '{model_name}' not found and pulling disabled.")
            return False
    except requests.exceptions.ConnectionError:
        print("⚠️ Ollama API not reachable at", host)
        return False


# 🧠 Configuration
DATA_DIR = "../markdown_docs"
COLLECTION_NAME = "markdown_rag"
QDRANT_HOST = "localhost"
QDRANT_PORT = 6333

# 1️⃣ Connect to Qdrant (persistent volume inside docker)
qdrant = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

# 2️⃣ Define embedding + LLM models
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

ensure_model("llama3.2:latest")
llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434")

# 3️⃣ Global LlamaIndex settings
Settings.embed_model = embed_model
Settings.llm = llm

# 4️⃣ Load Markdown documents recursively
print("📂 Loading markdown files...")
docs = SimpleDirectoryReader(
    input_dir=DATA_DIR,
    required_exts=[".md", ".yml", ".yaml"],
    recursive=True
).load_data()

# Add full file paths to metadata
for doc in docs:
    if "file_path" not in doc.metadata:
        doc.metadata["file_path"] = os.path.abspath(doc.metadata.get("file_name", ""))

print(f"✅ Loaded {len(docs)} markdown files.")

# 5️⃣ Ensure Qdrant collection exists with correct vector size
embedding_dim = len(embed_model.get_text_embedding("test"))

try:
    qdrant.get_collection(COLLECTION_NAME)
    print(f"✅ Using existing Qdrant collection: {COLLECTION_NAME}")
except Exception:
    print(f"🆕 Creating new Qdrant collection: {COLLECTION_NAME}")
    qdrant.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(size=embedding_dim, distance=models.Distance.COSINE),
    )

# 6️⃣ Create / reuse vector index using Qdrant
vector_store = QdrantVectorStore(client=qdrant, collection_name=COLLECTION_NAME)
index = VectorStoreIndex.from_documents(docs, vector_store=vector_store)

# 7️⃣ Query engine setup
query_engine = index.as_query_engine(similarity_top_k=3)

# query the docs

In [ ]:
# 8️⃣ Ask a question

question = """
gimme a full github action implementation for a erlang app
"""

response = query_engine.query(question)

print("🧠 Answer:\n", response.response)
print("\n📄 Sources:")
for src in response.source_nodes:
    print("-", src.metadata.get("file_path", "unknown"))
